In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize 
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.decomposition import NMF
from textblob import TextBlob
from spellchecker import SpellChecker
import emoji
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
from nltk.chunk import ne_chunk
import datetime

In [2]:
pip install pyspellchecker

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mishkaasli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df = pd.read_csv('Desktop/Bitcoin_tweets_updated.csv')

In [6]:
df

user_location  \
0     Bitcoin is Saving                       NaN   
1         Altcoin Daily           Los Angeles, CA   
2       Travladd Crypto       Manchester, England   
3         Altcoin Daily           Los Angeles, CA   
4         Investing.com                    Global   
...                 ...                       ...   
6950   Gate.io Exchange                       NaN   
6951       FABRI CRYPTO                       NaN   
6952    LadyCryptoGem 💎       Shilling/Promotion    
6953          Elja𝕭oom💥  World of Elja & #BoomBag   
6954       Rekt Capital                       NaN   

                                       user_description  user_created  \
0                                                   NaN   42749.53627   
1     🎥 Follow our YouTube channel for DAILY news & ...   43129.97579   
2     Partnered @Twitch | Crypto Enthusiast, Investo...   43530.37473   
3     🎥 Follow our YouTube channel for DAILY news & ...   43129.97579   
4     📈Real-time global financial market data, news,...   41247.63123   
...                                                 ...           ...   
6950  https://t.co/l3EgEt4lSx was created to enable ...   43004.20138   
6951  Crypto Investor💎 NFA. #BTC #BNB #ETH ❤️ I love...   42383.53295   
6952  Crypto Influencer | Grow your Crypto Projects ...   43702.67627   
6953  Founder and CEO of https://t.co/LLSurM7IBO | @...   42212.18962   
6954  Cryptocurrency Trader & Analyst\n\nAuthor of t...   43019.60738   

      user_followers  user_friends  user_favourites  user_verified  \
0             159956          6975           159298          False   
1             310145           358            20408          False   
2             128803           272            30581          False   
3             310145           358            20408          False   
4             496248           395             5884           True   
...              ...           ...              ...            ...   
6950          286799          2108             8489          False   
6951          487561         39522            36991           True   
6952          329030         33117               22          False   
6953          317362           871           214303          False   
6954          186727           196            36575          False   

      Unnamed: 8     date                                               text  \
0            NaN  5/29/21  “Corporate reserves are not for investing in s...   
1            NaN  5/29/21  #Bitcoin added to another public company's tre...   
2            NaN  5/29/21  Just a reminder #Bitcoin #btc https://t.co/uVI...   
3            NaN  5/29/21  Confirmed: #Bitcoin just taking a pit stop and...   
4            NaN  5/29/21  *Bitcoin Volume Surges Over $300 Million in Pa...   
...          ...      ...                                                ...   
6950         NaN  8/18/21  ✨https://t.co/8kWqgEeplL Now Offers Crypto Loa...   
6951         NaN  8/18/21  🥇First time in the history of BSC we are prese...   
6952         NaN  8/18/21  I'll hold #BTC #ETH #BNB #TRX and next bags #X...   
6953         NaN  8/18/21  U.S. Halted Dollar Shipments to Afghanistan to...   
6954         NaN  8/18/21  $SOL / #BTC - #sol #Solana \n\nSolana has rall...   

                                   hashtags                source  is_retweet  
0                 ['Bitcoin', 'BTC', 'BTC']    Twitter for iPhone       False  
1                        ['Bitcoin', 'BTC']       Twitter Web App       False  
2                        ['Bitcoin', 'btc']    Twitter for iPhone       False  
3             ['Bitcoin', 'BTC', 'Indy500']       Twitter Web App       False  
4                        ['BTC', 'Bitcoin']  Twitter Media Studio       False  
...                                     ...                   ...         ...  
6950                 ['BTC', 'ETH', 'USDT']                Buffer       False  
6951  ['BSCGem', 'Cake', 'BTC', 'Ethereum']    Twitter for iPhone       False  


In [7]:
filter = df['date'] > '6/1/21'
df = df[filter]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6870 entries, 85 to 6954
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0                     6870 non-null   object 
 1   user_location     5178 non-null   object 
 2   user_description  6729 non-null   object 
 3   user_created      6870 non-null   float64
 4   user_followers    6870 non-null   int64  
 5   user_friends      6870 non-null   int64  
 6   user_favourites   6870 non-null   int64  
 7   user_verified     6870 non-null   bool   
 8   Unnamed: 8        0 non-null      float64
 9   date              6870 non-null   object 
 10  text              6870 non-null   object 
 11  hashtags          6870 non-null   object 
 12  source            6870 non-null   object 
 13  is_retweet        6870 non-null   bool   
dtypes: bool(2), float64(2), int64(3), object(7)
memory usage: 711.2+ KB


In [9]:
sorted_list = (df.date.value_counts())
import matplotlib.pyplot as plt
sorted_list.iloc[15:25].plot(y='value',kind='bar')
plt.ylabel('total number of tweets')
plt.xlabel('date')
plt.title('# of Tweets Per Day')
plt.show()
sorted_list

7/21/21    477
6/22/21    419
7/20/21    377
7/23/21    371
8/9/21     352
7/30/21    349
6/21/21    332
7/22/21    320
6/23/21    285
8/20/21    267
8/19/21    264
7/19/21    244
8/16/21    235
8/18/21    226
8/17/21    217
7/5/21     183
7/24/21    180
8/8/21     174
7/4/21     151
7/2/21     148
7/18/21    147
7/3/21     145
7/26/21    141
8/15/21    137
7/17/21    117
8/14/21     88
7/28/21     80
7/27/21     73
8/6/21      64
8/5/21      58
6/20/21     50
7/29/21     45
8/21/21     39
7/25/21     39
8/7/21      38
7/16/21     33
8/4/21       5
Name: date, dtype: int64

In [10]:
df = df.drop(['user_location', 'Unnamed: 8',"user_description","user_created", "user_favourites","user_friends","user_verified", "source"], axis = 1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6870 entries, 85 to 6954
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0                   6870 non-null   object
 1   user_followers  6870 non-null   int64 
 2   date            6870 non-null   object
 3   text            6870 non-null   object
 4   hashtags        6870 non-null   object
 5   is_retweet      6870 non-null   bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 328.7+ KB


In [12]:
df

user_followers     date  \
85        Bitcoin Archive 🗄🚀🌔          347102  6/23/21   
86          Thread Reader App          477881  6/23/21   
87        CryptoCurrency News          315910  6/23/21   
88                  TheStreet          786675  6/23/21   
89    The Wolf Of All Streets          486343  6/23/21   
...                       ...             ...      ...   
6950         Gate.io Exchange          286799  8/18/21   
6951             FABRI CRYPTO          487561  8/18/21   
6952          LadyCryptoGem 💎          329030  8/18/21   
6953                Elja𝕭oom💥          317362  8/18/21   
6954             Rekt Capital          186727  8/18/21   

                                                   text  \
85    #Bitcoin and crypto exchange FTX is now a spon...   
86    @TheRealVSky Guten Tag, you can read it here: ...   
87    #Bitcoin is beneficial to everybody but most o...   
88    Stock Market Today: @jimcramer has a bit of im...   
89    The reaction when #Bitcoin finally breaks belo...   
...                                                 ...   
6950  ✨https://t.co/8kWqgEeplL Now Offers Crypto Loa...   
6951  🥇First time in the history of BSC we are prese...   
6952  I'll hold #BTC #ETH #BNB #TRX and next bags #X...   
6953  U.S. Halted Dollar Shipments to Afghanistan to...   
6954  $SOL / #BTC - #sol #Solana \n\nSolana has rall...   

                                   hashtags  is_retweet  
85                              ['Bitcoin']       False  
86                              ['Bitcoin']       False  
87                              ['Bitcoin']       False  
88                  ['Ethereum', 'Bitcoin']       False  
89                              ['Bitcoin']       False  
...                                     ...         ...  
6950                 ['BTC', 'ETH', 'USDT']       False  
6951  ['BSCGem', 'Cake', 'BTC', 'Ethereum']       False  
6952    ['BTC', 'ETH', 'BNB', 'TRX', 'XRP']       False  
6953                                ['BTC']       False  
6954     ['BTC', 'sol', 'Solana', 'Crypto']       False  

[6870 rows x 6 columns]

In [13]:
df = df.dropna()

In [14]:
df['index'] = df.index
df['text'] = df['text'].astype(str)
df['text'] = df['text'].replace('\n',' ', regex=True)

In [15]:
df['new_text1'] = [re.sub(r'(\s)#\w+', r'', x) for x in df['text']]
df['new_text2'] = [re.sub(r'(\s)@\w+', '', x) for x in df['new_text1']]
df.head(320)

user_followers     date  \
85              Bitcoin Archive 🗄🚀🌔          347102  6/23/21   
86                Thread Reader App          477881  6/23/21   
87              CryptoCurrency News          315910  6/23/21   
88                        TheStreet          786675  6/23/21   
89          The Wolf Of All Streets          486343  6/23/21   
..                              ...             ...      ...   
400                David Gokhshtein          398592  6/22/21   
401                          Okcoin          129053  6/22/21   
402  ‏محمد العرب...Mohammed Al arab          690095  6/22/21   
403                    Bitcoin News         1444205  6/22/21   
404                     AirdropStar          347621  6/22/21   

                                                  text  \
85   #Bitcoin and crypto exchange FTX is now a spon...   
86   @TheRealVSky Guten Tag, you can read it here: ...   
87   #Bitcoin is beneficial to everybody but most o...   
88   Stock Market Today: @jimcramer has a bit of im...   
89   The reaction when #Bitcoin finally breaks belo...   
..                                                 ...   
400  #Bitcoin look strong right now.   Keep pushing...   
401  Ready to get into the next @Stacks cycle and e...   
402  Stupidity made in China!! #Bitcoin mining is a...   
403  Jaime Guevara, a Salvadoran representative, ha...   
404  💧 All Games Token Airdrop 💧  🏆 Task:          ...   

                                              hashtags  is_retweet  index  \
85                                         ['Bitcoin']       False     85   
86                                         ['Bitcoin']       False     86   
87                                         ['Bitcoin']       False     87   
88                             ['Ethereum', 'Bitcoin']       False     88   
89                                         ['Bitcoin']       False     89   
..                                                 ...         ...    ...   
400                                        ['Bitcoin']       False    400   
401                                        ['Bitcoin']       False    401   
402          ['Bitcoin', 'America', 'Texas', 'Crypto']       False    402   
403                                        ['Bitcoin']       False    403   
404  ['cryptocurrency', 'BSC', 'Bitcoin', 'Ethereum...       False    404   

                                             new_text1  \
85   #Bitcoin and crypto exchange FTX is now a spon...   
86   @TheRealVSky Guten Tag, you can read it here: ...   
87   #Bitcoin is beneficial to everybody but most o...   
88   Stock Market Today: @jimcramer has a bit of im...   
89   The reaction when finally breaks below the ran...   
..                                                 ...   
400  #Bitcoin look strong right now.   Keep pushing...   
401  Ready to get into the next @Stacks cycle and e...   
402  Stupidity made in China!! mining is a good exa...   
403  Jaime Guevara, a Salvadoran representative, ha...   
404  💧 All Games Token Airdrop 💧  🏆 Task:          ...   

                                             new_text2  
85   #Bitcoin and crypto exchange FTX is now a spon...  
86   @TheRealVSky Guten Tag, you can read it here: ...  
87   #Bitcoin is beneficial to everybody but most o...  
88   Stock Market Today: has a bit of important adv...  
89   The reaction when finally breaks below the ran...  
..                                                 ...  
400  #Bitcoin look strong right now.   Keep pushing...  
401  Ready to get into the next cycle and earn rewa...  
402  Stupidity made in China!! mining is a good exa...  
403  Jaime Guevara, a Salvadoran representative, ha...  
404  💧 All Games Token Airdrop 💧  🏆 Task:          ...  

[320 rows x 9 columns]

In [16]:
df['new_text3'] = [re.sub(r'^#\w+', '', x) for x in df['new_text2']]
df['new_text4'] = [re.sub(r'^@\w+', '', x) for x in df['new_text3']]
df['new_text5'] = [re.sub(r'http\S+', '', x) for x in df['new_text4']]
df['new_text6'] = [re.sub(r'(\s)[$]\S+', '', x) for x in df['new_text5']]
df['new_text7'] = [re.sub(r'^[$]\S+', '', x) for x in df['new_text6']]

In [17]:
df['new_text8'] = [re.sub(r'\w*\d\w*', '', x) for x in df['new_text7']]
df['new_text9'] = [re.sub('[%s]' % re.escape(string.punctuation), '',x) for x in df['new_text8']]
df['new_text95'] = [re.sub(r'[^\w\s]', '', x) for x in df['new_text9']]
df['new_text10'] = [x.lower() for x in df['new_text95']]
df['new_text11'] = [emoji.get_emoji_regexp().sub('', x) for x in df['new_text10']]
df['new_text12'] = [re.sub(r'(\s).*?project\S+', '', x) for x in df['new_text11']]
df['new_text125'] = [x.strip() for x in df['new_text12']]
df['new_text1258'] = [x.lstrip(' ') for x in df['new_text125']]
df['new_text1259'] = [x.rstrip('…') for x in df['new_text1258']]
df['new_text13'] = [re.sub(r'[...]$', '', x) for x in df['new_text1259']]
df.new_text13.drop_duplicates()
df.head(50)

user_followers     date  \
85       Bitcoin Archive 🗄🚀🌔          347102  6/23/21   
86         Thread Reader App          477881  6/23/21   
87       CryptoCurrency News          315910  6/23/21   
88                 TheStreet          786675  6/23/21   
89   The Wolf Of All Streets          486343  6/23/21   
90          Bitcoin Magazine         1111558  6/23/21   
91                Randy Gage          150451  6/23/21   
92             CoinMarketCap         2073535  6/23/21   
93                  CoinDesk         1773404  6/23/21   
94               Steve Hanke          337896  6/23/21   
95          Bitcoin Magazine         1111560  6/23/21   
96       Bitcoin Archive 🗄🚀🌔          347105  6/23/21   
97                 TheStreet          786676  6/23/21   
98          Bitcoin Magazine         1111560  6/23/21   
99          BitMart.Exchange          194245  6/23/21   
100      Bitcoin Archive 🗄🚀🌔          347106  6/23/21   
101            Mike Schiemer          213776  6/23/21   
102          Kraken Exchange          722639  6/23/21   
103                 CoinDesk         1773405  6/23/21   
104      Bitcoin Archive 🗄🚀🌔          347106  6/23/21   
105         BitMart.Exchange          194246  6/23/21   
106         Bitcoin Magazine         1111563  6/23/21   
107          Peter McCormack          301348  6/23/21   
108            Ben Armstrong          431497  6/23/21   
109          Peter McCormack          301351  6/23/21   
110              Steve Hanke          337896  6/23/21   
111          Davinci Jeremie          206826  6/23/21   
112         NebraskanGooner📉          141270  6/23/21   
113              Nikkei Asia          251657  6/23/21   
114  The Wolf Of All Streets          486343  6/23/21   
115                Ash WSB 🍥          238529  6/23/21   
116                Ash WSB 🍥          238529  6/23/21   
117            CoinMarketCap         2073547  6/23/21   
118      Hürriyet Daily News          278893  6/23/21   
119                Adam Back          330444  6/23/21   
120                TheStreet          786678  6/23/21   
121                   reason          262995  6/23/21   
122            Altcoin Daily          352985  6/23/21   
123              AirdropStar          347606  6/23/21   
124                Ash WSB 🍥          238530  6/23/21   
125         NebraskanGooner📉          141271  6/23/21   
126                Ash WSB 🍥          238530  6/23/21   
127                Adam Back          330444  6/23/21   
128      CryptoCurrency News          315910  6/23/21   
129             Bitcoin News         1444008  6/23/21   
130                Adam Back          330445  6/23/21   
131            Cointelegraph         1048041  6/23/21   
132              JRNY Crypto          336421  6/23/21   
133                Ash WSB 🍥          238530  6/23/21   
134          Kraken Exchange          722643  6/23/21   

                                                  text  \
85   #Bitcoin and crypto exchange FTX is now a spon...   
86   @TheRealVSky Guten Tag, you can read it here: ...   
87   #Bitcoin is beneficial to everybody but most o...   
88   Stock Market Today: @jimcramer has a bit of im...   
89   The reaction when #Bitcoin finally breaks belo...   
90   #Bitcoin exchange @FTX_Official is now an offi...   
91   If you've ever wondered about how #bitcoin #cr...   
92   Word of the Day - Automated Market Maker  Anyo...   
93   Coming up on "All About Bitcoin":  • Marc Lopr...   
94   .@nayibbukele claims that by using #Bitcoin, i...   
95   China's #bitcoin mining crackdown has led to B...   
96   20,000 #Bitcoin miners cleaned and shipped out...   
97   .@Luke_Conway_ breaks down how a combination o...   
98   "If it needed more help than Square and Twitte...   
99   🔔 Did you know you can earn up to 30% commissi...   
100  Paraguay University will start accepting #Bitc...   
101  4 Benefits Of A VPN When Working With Cryptocu...   
102  Don’t miss Kraken’s Co-founder &amp; CEO @jesp...   
103  China's crackdo

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6870 entries, 85 to 6954
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0                   6870 non-null   object
 1   user_followers  6870 non-null   int64 
 2   date            6870 non-null   object
 3   text            6870 non-null   object
 4   hashtags        6870 non-null   object
 5   is_retweet      6870 non-null   bool  
 6   index           6870 non-null   int64 
 7   new_text1       6870 non-null   object
 8   new_text2       6870 non-null   object
 9   new_text3       6870 non-null   object
 10  new_text4       6870 non-null   object
 11  new_text5       6870 non-null   object
 12  new_text6       6870 non-null   object
 13  new_text7       6870 non-null   object
 14  new_text8       6870 non-null   object
 15  new_text9       6870 non-null   object
 16  new_text95      6870 non-null   object
 17  new_text10      6870 non-null   object
 18  new_tex

In [19]:
string_1 = (df.new_text13[479])
string_1.rstrip('…')
df['date']

85      6/23/21
86      6/23/21
87      6/23/21
88      6/23/21
89      6/23/21
         ...   
6950    8/18/21
6951    8/18/21
6952    8/18/21
6953    8/18/21
6954    8/18/21
Name: date, Length: 6870, dtype: object

In [20]:
df.new_text13

85      and crypto exchange ftx is now a sponsor of ma...
86      guten tag you can read it here  in todays delp...
87      is beneficial to everybody but most of all to ...
88      stock market today has a bit of important advi...
89      the reaction when finally breaks below the ran...
                              ...                        
6950    now offers crypto loans   borrow usdt at  dail...
6951    first time in the history of bsc we are presen...
6952                               ill hold and next bags
6953    us halted dollar shipments to afghanistan to k...
6954    solana has rallied to new all time highs yet a...
Name: new_text13, Length: 6870, dtype: object

In [21]:
df['tokenized_text'] = df['new_text13'].apply(word_tokenize) 
df.head(50)

user_followers     date  \
85       Bitcoin Archive 🗄🚀🌔          347102  6/23/21   
86         Thread Reader App          477881  6/23/21   
87       CryptoCurrency News          315910  6/23/21   
88                 TheStreet          786675  6/23/21   
89   The Wolf Of All Streets          486343  6/23/21   
90          Bitcoin Magazine         1111558  6/23/21   
91                Randy Gage          150451  6/23/21   
92             CoinMarketCap         2073535  6/23/21   
93                  CoinDesk         1773404  6/23/21   
94               Steve Hanke          337896  6/23/21   
95          Bitcoin Magazine         1111560  6/23/21   
96       Bitcoin Archive 🗄🚀🌔          347105  6/23/21   
97                 TheStreet          786676  6/23/21   
98          Bitcoin Magazine         1111560  6/23/21   
99          BitMart.Exchange          194245  6/23/21   
100      Bitcoin Archive 🗄🚀🌔          347106  6/23/21   
101            Mike Schiemer          213776  6/23/21   
102          Kraken Exchange          722639  6/23/21   
103                 CoinDesk         1773405  6/23/21   
104      Bitcoin Archive 🗄🚀🌔          347106  6/23/21   
105         BitMart.Exchange          194246  6/23/21   
106         Bitcoin Magazine         1111563  6/23/21   
107          Peter McCormack          301348  6/23/21   
108            Ben Armstrong          431497  6/23/21   
109          Peter McCormack          301351  6/23/21   
110              Steve Hanke          337896  6/23/21   
111          Davinci Jeremie          206826  6/23/21   
112         NebraskanGooner📉          141270  6/23/21   
113              Nikkei Asia          251657  6/23/21   
114  The Wolf Of All Streets          486343  6/23/21   
115                Ash WSB 🍥          238529  6/23/21   
116                Ash WSB 🍥          238529  6/23/21   
117            CoinMarketCap         2073547  6/23/21   
118      Hürriyet Daily News          278893  6/23/21   
119                Adam Back          330444  6/23/21   
120                TheStreet          786678  6/23/21   
121                   reason          262995  6/23/21   
122            Altcoin Daily          352985  6/23/21   
123              AirdropStar          347606  6/23/21   
124                Ash WSB 🍥          238530  6/23/21   
125         NebraskanGooner📉          141271  6/23/21   
126                Ash WSB 🍥          238530  6/23/21   
127                Adam Back          330444  6/23/21   
128      CryptoCurrency News          315910  6/23/21   
129             Bitcoin News         1444008  6/23/21   
130                Adam Back          330445  6/23/21   
131            Cointelegraph         1048041  6/23/21   
132              JRNY Crypto          336421  6/23/21   
133                Ash WSB 🍥          238530  6/23/21   
134          Kraken Exchange          722643  6/23/21   

                                                  text  \
85   #Bitcoin and crypto exchange FTX is now a spon...   
86   @TheRealVSky Guten Tag, you can read it here: ...   
87   #Bitcoin is beneficial to everybody but most o...   
88   Stock Market Today: @jimcramer has a bit of im...   
89   The reaction when #Bitcoin finally breaks belo...   
90   #Bitcoin exchange @FTX_Official is now an offi...   
91   If you've ever wondered about how #bitcoin #cr...   
92   Word of the Day - Automated Market Maker  Anyo...   
93   Coming up on "All About Bitcoin":  • Marc Lopr...   
94   .@nayibbukele claims that by using #Bitcoin, i...   
95   China's #bitcoin mining crackdown has led to B...   
96   20,000 #Bitcoin miners cleaned and shipped out...   
97   .@Luke_Conway_ breaks down how a combination o...   
98   "If it needed more help than Square and Twitte...   
99   🔔 Did you know you can earn up to 30% commissi...   
100  Paraguay University will start accepting #Bitc...   
101  4 Benefits Of A VPN When Working With Cryptocu...   
102  Don’t miss Kraken’s Co-founder &amp; CEO @jesp...   
103  China's crackdo

In [22]:
filter_1 = (df['date'] == '8/8/21')
df_1 = df[filter_1]
df_1.head(50)

user_followers    date  \
5108                     Benjamin Cowen          224031  8/8/21   
5109                    Wealth Theory ™          198973  8/8/21   
5110                     Benjamin Cowen          224031  8/8/21   
5111                     Benjamin Cowen          224031  8/8/21   
5112                       Preston Pysh          248328  8/8/21   
5113                    Wealth Theory ™          198973  8/8/21   
5114                  Bitcoin is Saving          183248  8/8/21   
5115                   David Gokhshtein          408179  8/8/21   
5116                              Payne          194343  8/8/21   
5117                     Benjamin Cowen          224031  8/8/21   
5118                         Lark Davis          440276  8/8/21   
5119                 BITCOIN MAXIMALIST          360254  8/8/21   
5120                      Altcoin Daily          426686  8/8/21   
5121                     Benjamin Cowen          224031  8/8/21   
5122                     Benjamin Cowen          224031  8/8/21   
5123            The Wolf Of All Streets          520750  8/8/21   
5124                      Altcoin Daily          426686  8/8/21   
5125                         Lark Davis          440276  8/8/21   
5126                         Lark Davis          440275  8/8/21   
5127                     Benjamin Cowen          224031  8/8/21   
5128                      Rebecca Grant          165001  8/8/21   
5129                      Altcoin Daily          426688  8/8/21   
5130                  Bitcoin is Saving          183248  8/8/21   
5131                   NebraskanGooner📈          149812  8/8/21   
5132                   David Gokhshtein          408179  8/8/21   
5133                   Robert Breedlove          150239  8/8/21   
5134                   Jimmy Song (송재준)          256872  8/8/21   
5135                   Robert Breedlove          150239  8/8/21   
5136                        Trish Regan          684353  8/8/21   
5137                   David Gokhshtein          408180  8/8/21   
5138                   David Gokhshtein          408180  8/8/21   
5139  Jason A. Williams “Parabolic Guy”          139850  8/8/21   
5140               Michaël van de Poppe          365316  8/8/21   
5141                  Bitcoin is Saving          183247  8/8/21   
5142                Bloomberg Quicktake         1252588  8/8/21   
5143                   Bitcoin Magazine         1229658  8/8/21   
5144                           The Moon          417303  8/8/21   
5145  Jason A. Williams “Parabolic Guy”          139850  8/8/21   
5146                       Rekt Capital          179771  8/8/21   
5147             👑 Shelby 👑 SXP to 100$          197851  8/8/21   
5148             👑 Shelby 👑 SXP to 100$          197852  8/8/21   
5149                       Rekt Capital          179771  8/8/21   
5150                Bitcoin Archive 🗄🚀🌔          413398  8/8/21   
5151                       Rekt Capital          179771  8/8/21   
5152               Michaël van de Poppe          365316  8/8/21   
5153                Geoff De Weaver 🇺🇸🦅          371543  8/8/21   
5154                       Rekt Capital          179772  8/8/21   
5155                   David Gokhshtein          408180  8/8/21   
5156                   David Gokhshtein          408180  8/8/21   
5157            The Wolf Of All Streets          520764  8/8/21   

                                                   text  \
5108  As we come into the weekly close, #Bitcoin is ...   
5109  I’m not sure that people have grasped the fina...   
5110  15 minutes to go...  Will #Bitcoin close above...   
5111  All the "market cycles" of insane price apprec...   
5112  @JStein_WaPo Jeff here’s the story: the #Bitco...   
5113  Gold breaks under $1700 amidst unprecedented f...   
5114  Sell your real estate investments, buy #bitcoi...   
5115  Me : when #Bitcoin and $ETH start going parabo...   
5116  Peter Schiff complains that #Bitcoin is too vo...   
5117  You should use historical m

In [23]:
df.to_csv('preprocessing_df.csv')